# Run five solvers on 20 versions of the (s, S) inventory problem.

Produces plots appearing in the INFORMS Journal on Computing submission.

## Initial Setup

Import the necessary libraries and modules for the data farming experiment.

In [ ]:
import sys
from pathlib import Path

# Get the current working directory
# This should be where the .ipynb file itself resides when launched
current_dir = Path.cwd()

# Construct the path to the directory containing 'simopt'
# This assumes 'simopt' is one level up from the current directory
simopt_parent_dir = current_dir.parent

# Convert the simopt folder Path object to a string and add to sys.path
sys.path.append(str(simopt_parent_dir))

from simopt.experiment_base import (  # noqa: E402
    PlotType,
    ProblemSolver,
    plot_area_scatterplots,
    plot_progress_curves,
    plot_solvability_profiles,
    plot_terminal_progress,
    plot_terminal_scatterplots,
    post_normalize,
)

In [ ]:
# Classes for Problem and Solver info
# These don't need modified, they're just used to organize the problem and solver info.
class Problem:
    """Problem class to hold problem information."""

    def __init__(
        self,
        name: str,
        rename: str | None = None,
        fixed_factors: dict | None = None,
        model_fixed_factors: dict | None = None,
    ) -> None:
        """Initialize the Problem with name, rename, and problem/model fixed factors."""
        self.name = name
        self.rename = rename if rename else name
        self.fixed_factors = fixed_factors if fixed_factors else {}
        self.model_fixed_factors = model_fixed_factors if model_fixed_factors else {}


class Solver:
    """Solver class to hold solver information."""

    def __init__(
        self, name: str, rename: str | None = None, fixed_factors: dict | None = None
    ) -> None:
        """Initialize the Solver with name, rename, and solver fixed factors."""
        self.name = name
        self.rename = rename if rename else name
        self.fixed_factors = fixed_factors if fixed_factors else {}

In [ ]:
# Experiment Configuration
num_macroreps = 10
num_postreps = 100  # Using 100 results in high variance in plot_area_scatterplots?
num_postnorms = 200


# Function to run an experiment with the given problems and solvers.
def run_experiment(
    problems: list[Problem],
    solvers: list[Solver],
) -> list[list[ProblemSolver]]:
    """Run the Experiment for the given problems and solvers.

    Args:
        problems: List of Problem instances.
        solvers: List of Solver instances.

    Returns:
        List[list[ProblemSolver]]: A list of lists containing ProblemSolver instances,
        grouped by problem.
    """
    all_experiments = []
    for problem_idx, problem in enumerate(problems):
        print(
            f"Running Problem {problem_idx + 1}/{len(problems)}: {problem.rename}...",
            end="",
            flush=True,
        )
        # Keep track of experiments on the same problem for post-processing.
        experiments_same_problem = []
        # Create each ProblemSolver and run it.
        for solver in solvers:
            new_experiment = ProblemSolver(
                solver_name=solver.name,
                solver_rename=solver.rename,
                solver_fixed_factors=solver.fixed_factors,
                problem_name=problem.name,
                problem_rename=problem.rename,
                problem_fixed_factors=problem.fixed_factors,
                model_fixed_factors=problem.model_fixed_factors,
            )
            # Run and post-replicate the experiment.
            new_experiment.run(n_macroreps=num_macroreps)
            new_experiment.post_replicate(n_postreps=num_postreps)
            experiments_same_problem.append(new_experiment)

        # Post-normalize experiments with L.
        # Provide NO proxies for f(x0), f(x*), or f(x).
        post_normalize(
            experiments=experiments_same_problem,
            n_postreps_init_opt=num_postnorms,
        )
        all_experiments.append(experiments_same_problem)
        print("Done.")
    print("All experiments completed.")
    return all_experiments

In [ ]:
# Solvers to use in the experiment.
# Includes two versions of random search with varying sample sizes.
# The rename will be used in the plots to differentiate them.
solvers = [
    Solver(name="RNDSRCH", rename="RNDSRCH_ss=10", fixed_factors={"sample_size": 10}),
    Solver(name="RNDSRCH", rename="RNDSRCH_ss=50", fixed_factors={"sample_size": 50}),
    Solver(name="ASTRODF"),
    Solver(name="NELDMD"),
    Solver(name="STRONG"),
]

In [ ]:
# Configure the problem
demand_means = [25.0, 50.0, 100.0, 200.0, 400.0]
lead_means = [1.0, 3.0, 6.0, 9.0]

# Create all the problem variants.
SSCONT_problems = [
    Problem(
        name="SSCONT-1",
        rename=f"SSCONT-1_dm={dm}_lm={lm}",
        fixed_factors={"budget": 1000},
        model_fixed_factors={
            "demand_mean": dm,
            "lead_mean": lm,
        },
    )
    for dm in demand_means
    for lm in lead_means
]

# Run the experiment for SSCONT problems.
SSCONT_experiments = run_experiment(
    problems=SSCONT_problems,
    solvers=solvers,
)

In [ ]:
# Combine the experiments into a list of lists, where the outer list
# contains all the experiments for a single solver, and the inner lists
# contain the ProblemSolver instances associated with that solver.

experiment_dict = {}
for exp_problem_list in SSCONT_experiments:
    for experiment in exp_problem_list:
        # Use the solver name as the key and append the ProblemSolver instance.
        key = experiment.solver.name
        if key not in experiment_dict:
            experiment_dict[key] = []
        experiment_dict[key].append(experiment)
# Turn the dictionary into a list of lists.
experiments = list(experiment_dict.values())

In [ ]:
# PLOTTING SETTINGS
enable_confidence_intervals = True
alpha = 0.2

In [ ]:
plot_area_scatterplots(
    experiments, all_in_one=True, plot_conf_ints=True, print_max_hw=True
)

In [ ]:
plot_solvability_profiles(
    experiments,
    plot_type=PlotType.CDF_SOLVABILITY,
    solve_tol=0.1,
    all_in_one=True,
    plot_conf_ints=True,
    print_max_hw=True,
)

In [ ]:
plot_solvability_profiles(
    experiments,
    plot_type=PlotType.QUANTILE_SOLVABILITY,
    solve_tol=0.1,
    beta=0.5,
    all_in_one=True,
    plot_conf_ints=True,
    print_max_hw=True,
)

In [ ]:
plot_solvability_profiles(
    experiments=experiments,
    plot_type=PlotType.DIFF_CDF_SOLVABILITY,
    solve_tol=0.1,
    ref_solver="ASTRODF",
    all_in_one=True,
    plot_conf_ints=True,
    print_max_hw=True,
)

In [ ]:
plot_solvability_profiles(
    experiments=experiments,
    plot_type=PlotType.DIFF_QUANTILE_SOLVABILITY,
    solve_tol=0.1,
    beta=0.5,
    ref_solver="ASTRODF",
    all_in_one=True,
    plot_conf_ints=True,
    print_max_hw=True,
)

In [ ]:
plot_terminal_scatterplots(experiments, all_in_one=True)

In [ ]:
n_problems = len(SSCONT_problems)
n_solvers = len(experiments)

for i in range(n_problems):
    plot_progress_curves(
        [experiments[solver_idx][i] for solver_idx in range(n_solvers)],
        plot_type=PlotType.MEAN,
        all_in_one=True,
        plot_conf_ints=True,
        print_max_hw=True,
        normalize=False,
    )
    plot_terminal_progress(
        [experiments[solver_idx][i] for solver_idx in range(n_solvers)],
        plot_type=PlotType.VIOLIN,
        normalize=True,
        all_in_one=True,
    )

In [ ]:
# from math import log, sqrt
# import matplotlib.pyplot as plt
# import numpy as np
# for mu_d in demand_means:
#     for mu_l in lead_means:
#         lq = mu_d * mu_l / 3
#         uq = mu_d * mu_l + 2 * sqrt(2 * mu_d**2 * mu_l)
#         mu = (log(lq) + log(uq)) / 2
#         sigma = (log(uq) - mu) / 1.96
#         print(
#             round(mu_d, 0),
#             round(mu_l, 0),
#             round(lq, 2),
#             round(uq, 2),
#             round(mu, 2),
#             round(sigma, 2),
#         )
#         s = np.random.lognormal(mu, sigma, 1000)
#         plt.hist(
#             s,
#             density=True,
#             alpha=0.5,
#             label=str(round(mu_d, 0)) + "," + str(round(mu_l, 0)),
#             bins=50,
#             color="blue",
#         )
#         plt.axvline(lq, color="red")
#         plt.axvline(uq, color="red")
#         plt.axis("tight")
#         plt.legend()
#         plt.show()

# # Mean progress curves from all solvers on one problem. Problem instance 0.
# plot_progress_curves(
#     experiments=[experiments[solver_idx][0] for solver_idx in range(n_solvers)],
#     plot_type="mean",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Mean progress curves from all solvers on one problem. Problem instance 22.
# plot_progress_curves(
#     experiments=[experiments[solver_idx][22] for solver_idx in range(n_solvers)],
#     plot_type="mean",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Plot 0.9-quantile progress curves from all solvers on one problem.
# # Problem instance 0.
# plot_progress_curves(
#     experiments=[experiments[solver_idx][0] for solver_idx in range(n_solvers)],
#     plot_type="quantile",
#     beta=0.9,
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Plot 0.9-quantile progress curves from all solvers on one problem.
# # Problem instance 22.
# plot_progress_curves(
#     experiments=[experiments[solver_idx][22] for solver_idx in range(n_solvers)],
#     plot_type="quantile",
#     beta=0.9,
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Plot cdf of 0.2-solve times for all solvers on one problem.
# # Problem instance 0.
# plot_solvability_cdfs(
#     experiments=[experiments[solver_idx][0] for solver_idx in range(n_solvers)],
#     solve_tol=0.2,
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Plot cdf of 0.2-solve times for all solvers on one problem.
# # Problem instance 22.
# plot_solvability_cdfs(
#     experiments=[experiments[solver_idx][22] for solver_idx in range(n_solvers)],
#     solve_tol=0.2,
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
# )

# # Plot area scatterplots of all solvers on all problems.
# plot_area_scatterplots(
#     experiments=experiments, all_in_one=True, plot_CIs=False, print_max_hw=False
# )

# # Plot cdf 0.1-solvability profiles of all solvers on all problems.
# plot_solvability_profiles(
#     experiments=experiments,
#     plot_type="cdf_solvability",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
#     solve_tol=0.1,
# )

# # Plot 0.5-quantile 0.1-solvability profiles of all solvers on all problems.
# plot_solvability_profiles(
#     experiments=experiments,
#     plot_type="quantile_solvability",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
#     solve_tol=0.1,
#     beta=0.5,
# )

# # Plot difference of cdf 0.1-solvability profiles of all solvers on all problems.
# # Reference solver = ASTRO-DF.
# plot_solvability_profiles(
#     experiments=experiments,
#     plot_type="diff_cdf_solvability",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
#     solve_tol=0.1,
#     ref_solver="ASTRO-DF",
# )

# # Plot difference of 0.5-quantile 0.1-solvability profiles of all solvers on
# # all problems.
# # Reference solver = ASTRO-DF.
# plot_solvability_profiles(
#     experiments=experiments,
#     plot_type="diff_quantile_solvability",
#     all_in_one=True,
#     plot_CIs=True,
#     print_max_hw=False,
#     solve_tol=0.1,
#     beta=0.5,
#     ref_solver="ASTRO-DF",
# )